In [2]:
import pystac_client
import planetary_computer

import duckdb
import adlfs

In [25]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
search = catalog.search(collections=["gbif"])
items = search.get_all_items()
items = {x.id: x for x in items}

# most recent snapshot (first item)
item = list(items.values())[0]

In [28]:

keys = item.assets["data"].extra_fields["table:storage_options"]
AZURE_STORAGE_ACCOUNT_NAME = keys["account_name"] 
AZURE_STORAGE_ACCOUNT_KEY = keys["credential"]

In [29]:

fs = adlfs.AzureBlobFileSystem(account_name=AZURE_STORAGE_ACCOUNT_NAME, account_key=AZURE_STORAGE_ACCOUNT_KEY )


In [30]:
con = duckdb.connect()
con.register_filesystem(fs)
df = con.execute(f'''
    SELECT *
    FROM read_parquet("abfs://gbif/occurrence/2023-02-01/occurrence.parquet/*")
    LIMIT 10
    '''
).df()



df

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

,gbifid,datasetkey,occurrenceid,kingdom,phylum,class,order,family,genus,species,...,identifiedby,dateidentified,license,rightsholder,recordedby,typestatus,establishmentmeans,lastinterpreted,mediatype,issue
0,3873673048,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/12667...,Animalia,Arthropoda,Insecta,Odonata,Libellulidae,Sympetrum,Sympetrum sanguineum,...,[Александр Гончаров],2022-07-17 17:37:45,CC_BY_NC_4_0,Александр Гончаров,[Александр Гончаров],[],NaN,2023-01-30 15:23:17.970,[StillImage],[COORDINATE_ROUNDED]
1,3892292465,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/12674...,Plantae,Tracheophyta,Magnoliopsida,Lamiales,Orobanchaceae,Castilleja,Castilleja rhexifolia,...,[emilyschoi],2022-07-18 01:29:06,CC_BY_NC_4_0,emilyschoi,[emilyschoi],[],NaN,2023-01-30 15:23:18.181,[StillImage],[COORDINATE_ROUNDED]
2,3874183310,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/12683...,Animalia,Arthropoda,Insecta,Lepidoptera,Lycaenidae,Lycaena,Lycaena virgaureae,...,[Rikke Anker Jensen],2022-07-18 16:56:35,CC_BY_NC_4_0,Rikke Anker Jensen,[Rikke Anker Jensen],[],NaN,2023-01-30 15:23:18.637,[StillImage],[COORDINATE_ROUNDED]
3,3874195286,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/12692...,Plantae,Tracheophyta,Magnoliopsida,Solanales,Solanaceae,Cestrum,Cestrum elegans,...,[Reiner Richter],2022-07-19 06:04:52,CC_BY_4_0,Reiner Richter,[Reiner Richter],[],NaN,2023-01-30 15:23:18.812,[StillImage],[COORDINATE_ROUNDED]
4,3873688536,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/12702...,Animalia,Arthropoda,Insecta,Lepidoptera,Nymphalidae,Melanargia,Melanargia galathea,...,[Anna Nikolenko],2022-07-19 23:07:51,CC_BY_4_0,Anna Nikolenko,[Anna Nikolenko],[],NaN,2023-01-30 15:23:19.000,[StillImage],[COORDINATE_ROUNDED]
5,3874142613,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/12722...,Animalia,Chordata,Aves,Charadriiformes,Recurvirostridae,Himantopus,Himantopus himantopus,...,[Stefano],2022-07-21 14:27:28,CC_BY_NC_4_0,Stefano,[Stefano],[],NaN,2023-01-30 15:23:19.343,[StillImage],[COORDINATE_ROUNDED]
6,3873540748,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/12772...,Plantae,Tracheophyta,Magnoliopsida,Caryophyllales,Caryophyllaceae,Dianthus,Dianthus borbasii,...,[Aleksey Baushev],2022-07-24 18:38:41,CC0_1_0,Aleksey Baushev,[Aleksey Baushev],[],NaN,2023-01-30 15:23:19.510,[StillImage],[COORDINATE_ROUNDED]
7,3881606584,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/12809...,Animalia,Chordata,Testudines,NaN,Chelydridae,Chelydra,Chelydra serpentina,...,[Danny Nelson],2022-07-27 02:50:45,CC_BY_NC_4_0,Danny Nelson,[Danny Nelson],[],NaN,2023-01-30 15:23:19.789,[StillImage],[COORDINATE_ROUNDED]
8,3881342737,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/12850...,Animalia,Arthropoda,Insecta,Odonata,Coenagrionidae,Enallagma,Enallagma cyathigerum,...,[winu],2022-07-29 20:02:13,CC_BY_NC_4_0,winu,[winu],[],NaN,2023-01-30 15:23:19.958,[StillImage],[COORDINATE_ROUNDED]
9,3881470820,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/12900...,Animalia,Arthropoda,Insecta,Lepidoptera,Geometridae,Leptostales,Leptostales pannaria,...,[Mary Spolyar],2022-08-01 17:01:50,CC_BY_NC_4_0,Mary Spolyar,[Mary Spolyar],[],NaN,2023-01-30 15:23:20.162,[StillImage],[COORDINATE_ROUNDED]


In [ ]:

df = con.execute(f'''
    SELECT genus, species, COUNT(*) AS n
    FROM read_parquet("abfs://gbif/occurrence/2023-02-01/occurrence.parquet/*")
    GROUP BY genus, species
    ORDER BY n DESC
    LIMIT 10
    '''
).df()

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

In [34]:
# Direct S3 query
con.execute("INSTALL httpfs")
con.execute("LOAD httpfs")

dd = con.execute(f'''
    SELECT genus, species, COUNT(*) AS n
    FROM read_parquet("s3://gbif-open-data-us-east-1/occurrence/2023-02-01/occurrence.parquet/*")
    GROUP BY genus, species
    ORDER BY n DESC
    LIMIT 10
    '''
).df()

dd

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

,genus,species,n
0,NaN,NaN,57336401
1,Anas,Anas platyrhynchos,22248266
2,Zenaida,Zenaida macroura,19548024
3,Corvus,Corvus brachyrhynchos,18294645
4,Turdus,Turdus migratorius,18231670
5,Sturnus,Sturnus vulgaris,17975584
6,Branta,Branta canadensis,17275909
7,Cardinalis,Cardinalis cardinalis,16723452
8,Passer,Passer domesticus,15378488
9,Cyanocitta,Cyanocitta cristata,14997848
